# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import csv data and create a dataframe.
cities_df = pd.read_csv('../WeatherPy/data/city_weather.csv')
# Drop the NA that were inserted for cities without weather info
cities_df.dropna(inplace=True)

In [3]:
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hearst,49.700000,-83.666667,46.49,98.0,100.0,5.03,CA,1.600225e+09
1,Namatanai,-3.666667,152.433333,83.01,76.0,71.0,10.94,PG,1.600225e+09
2,Noumea,-22.266667,166.450000,78.80,47.0,90.0,9.17,NC,1.600225e+09
3,Ushuaia,-54.800000,-68.300000,35.60,100.0,40.0,9.17,AR,1.600225e+09
4,Nikolskoye,47.762400,46.406700,46.99,93.0,94.0,4.47,RU,1.600225e+09
...,...,...,...,...,...,...,...,...,...
994,Warrnambool,-38.381760,142.487991,64.99,63.0,100.0,8.99,AU,1.600225e+09
995,Tiassale,5.898394,-4.822926,72.72,93.0,99.0,2.89,CI,1.600225e+09
996,Algiers,36.763056,3.050556,66.20,88.0,20.0,3.36,DZ,1.600225e+09
997,Werda,-25.266667,23.283333,61.77,27.0,62.0,8.57,BW,1.600225e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
coordinates = cities_df[['Lat','Lng']]

# store the figure in a variable so we can continue to add layers to the same map
fig = gmaps.figure()


# Store 'Poverty Rate' in a variable called 'poverty_rate'
humidity_data = cities_df['Humidity'].astype(float)

# Create the heatmap layer using locations and poverty_rate
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity_data, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=0.9)
# Add heatmap layer to figure
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

weather_criteria_df = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70) & 
              (cities_df['Wind Speed'] < 10) & 
             (cities_df['Cloudiness'] == 0)]

print(f"The list of cities matching the criteria contains {len(weather_criteria_df)} cities.")
weather_criteria_df.reset_index(drop=True, inplace=True)
weather_criteria_df

The list of cities matching the criteria contains 24 cities.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.120278,-134.969167,74.07,71.0,0.0,0.40,PF,1.600225e+09
1,Tahta,26.768056,31.493889,78.80,39.0,0.0,6.93,EG,1.600225e+09
2,Nouadhibou,20.900000,-17.066667,76.46,77.0,0.0,9.82,MR,1.600225e+09
3,Gulariya,28.233333,81.333333,78.01,92.0,0.0,4.59,NP,1.600225e+09
4,Barra Do Garcas,-15.883333,-52.250000,75.18,30.0,0.0,2.46,BR,1.600225e+09
5,Charsadda,34.143453,71.731731,78.80,69.0,0.0,4.70,PK,1.600225e+09
6,Komsomolskiy,69.166667,172.700000,71.60,30.0,0.0,6.93,RU,1.600225e+09
7,Puerto Maldonado,-12.600000,-69.183333,71.53,86.0,0.0,2.73,PE,1.600225e+09
8,Kitob,39.118056,66.875556,71.92,21.0,0.0,1.81,UZ,1.600225e+09
9,Sile,41.175403,29.613330,75.00,69.0,0.0,8.01,TR,1.600225e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_criteria_df

#### Setup the framework for the API call

* We are going to narrow the search for the first 10 cities.

In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in weather_criteria_df.iloc[0:10,:].iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    print(f"Searching within {row['City']}, {row['Country']}...")

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)


    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Searching within Rikitea, PF...


/Users/gabrielpivaro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/gabrielpivaro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Searching within Tahta, EG...
Searching within Nouadhibou, MR...
Searching within Gulariya, NP...
Searching within Barra Do Garcas, BR...
Searching within Charsadda, PK...
Searching within Komsomolskiy, RU...
Missing field/result... skipping.
Searching within Puerto Maldonado, PE...
Searching within Kitob, UZ...
Searching within Sile, TR...


In [8]:
# Drop the rows that contains NA leaving only the 10 hotels in the list with the desired columns
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Hotel Name']].dropna()
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Rikitea,PF,-23.120278,-134.969167,Pension Maro'i
1,Tahta,EG,26.768056,31.493889,انس رشوان
2,Nouadhibou,MR,20.900000,-17.066667,El Medina
3,Gulariya,NP,28.233333,81.333333,Krishnasar Resort
4,Barra Do Garcas,BR,-15.883333,-52.250000,Hotel Pousada Tropical
5,Charsadda,PK,34.143453,71.731731,SHAHRIYAR KHAN HASTNAGAR
7,Puerto Maldonado,PE,-12.600000,-69.183333,Wasai Hotel
8,Kitob,UZ,39.118056,66.875556,KITOB SAYYOH
9,Sile,TR,41.175403,29.613330,Seref Hotel


In [9]:
# Export the hotels dataframe as csv
hotel_df.to_csv('data/hotels_list.csv',index=False)

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))